In [1]:
import tensorflow as tf
import os

# This is the basic shape for an inference model code.

In [2]:
# First initialize variables/model parameters
W = tf.Variable(tf.zeros([2, 1]), name="weights")
b = tf.Variable(0., name="bias")

In [3]:
# define the training loop operations
def inputs():
    # read/generate input training data X and expected outputs Y
    weight_age = [
        [84, 46], [73, 20], [65, 52], [70, 30], [76, 57], [69, 25], [63, 28], [72, 36], [79, 57], [75, 44],
        [27, 24], [89, 31], [65, 52], [57, 23], [59, 60], [69, 48], [60, 34], [79, 51], [75, 50], [82, 34]]
    blood_fat_content = [354, 190, 405, 263, 451, 302, 288, 385, 402, 365,
                         209, 290, 346, 254, 395, 434, 220, 374, 308, 220]
    return tf.to_float(weight_age), tf.to_float(blood_fat_content)

In [4]:
def inference(X):
    # compute inference model over data X and return the result
    return tf.matmul(X, W) + b

In [5]:
def loss(X, Y):
    # compute loss over training data X and expected outputs Y
    Y_predicted = inference(X)
    return tf.reduce_sum(tf.squared_difference(Y, Y_predicted))

In [6]:
def train(total_loss):
    # train / adjust model parameters according to computed total loss
    learning_rate = 0.0000001
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

In [7]:
def evaluate(sess, X, Y):
    # evaluate the resulting trained model
    print(sess.run(inference([[80., 25.]]))) # ~ 303
    print(sess.run(inference([[65., 25.]]))) # ~ 256

In [8]:
# Create a saver.
saver = tf.train.Saver()

In [ ]:
# Launch the graph in a session, setup boilerplate
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    initial_step = 0
    # verify if we don't have a checkpoint saved already
    checkpoint = tf.train.get_checkpoint_state(os.path.dirname(os.path.realpath('__file__')))
    if checkpoint and checkpoint.model_checkpoint_path:
        # Restores from checkpoint
        saver.restore(sess, checkpoint.model_checkpoint_path)
        initial_step = int(checkpoint.model_checkpoint_path.rsplit('-', 1)[1])

    X, Y = inputs()

    total_loss = loss(X, Y)
    train_op = train(total_loss)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # actual training loop
    training_steps = 1000
    print(initial_step, initial_step + training_steps)
    for step in range(initial_step, initial_step + training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 10 == 0:
            print("loss: ", sess.run([total_loss]))
    
    evaluate(sess, X, Y)

    saver.save(sess, 'ml-basics-model', global_step=initial_step + training_steps)
    
    coord.request_stop()
    coord.join(threads)
    sess.close()